In [ ]:
from rgb import process_images_and_save_csv

# Paths
folder_path = "RAW"
csv_save_path = "Res/Images_seethru_rawRGB_HSV_ST_raw_all_values.csv"

process_images_and_save_csv(folder_path, csv_save_path)

########  To choose Module 1 ###############
## If Avg_R < ( Avg_G/5 | Avg_B/5):
#      # Low-lit -> IIluminateNet
# Else:
#      # High-lit-> SEF


Per-image RGB and HSV values saved to: Res/Images_seethru_rawRGB_HSV_ST_raw_all_values.csv

===== FINAL AVERAGE (ALL IMAGES) =====
Average RGB: R=4.30, G=24.79, B=17.39
Average HSV: H=80.27, S=211.38, V=24.79
Final averages appended to CSV.


CHECK FOR RGB CONDITION

MODULE 1

RUN ILLUMINATENET IF CONDITION IS SATISFIED

In [3]:
from transmap_generate import process_images 

process_images(input_dir="RAW", 
               output_dir="Transmap")

In [5]:
##  TRAINING
!python3 illuminatenet_main.py --images "RAW" \
                     --depth "Transmap" \
                     --output "IlluminateNet_output" \
                     --checkpoints "IlluminateNet_output/check"



Seed: 4709336956823664384
training
100%|████████████████████████████████████| 500/500 [00:00<00:00, 3956890.57it/s]
Avg time per eval: 14.244536 ms (14.244531 ms rs, 0.000005 ms at)
Losses: 0.076440647
UCIQE: 1.0060245204218927
UQIMS: 2.9031435836405346
UCIQE: 0.8318632931731588
UQIMS: 3.35717304036756
UCIQE: 0.8577964065195551
UQIMS: 3.370247867034773
UCIQE: 0.6263992775275957
UQIMS: 3.301056507461425
UCIQE: 0.6422951634800363
UQIMS: 3.1749169514662956
UCIQE: 0.577466514526276
UQIMS: 3.1968348031768015
UCIQE: 0.6845659151579238
UQIMS: 3.30644705172286
Evaluation metrics saved to IlluminateNet_output/evaluation_metrics.xlsx


In [3]:
##  INFERENCE
!python3 illuminatenet_inf.py --images "RAW" \
                     --depth "Transmap" \
                     --output 'IlluminateNet_inf' \
                     --checkpoints 'IlluminateNet_output/check'

ELSE RUN SEF

In [7]:
import cv2 as cv
import os
from aocm_algorithm import spectral_equalization_filter

# --- Paths ---
input_folder = 'RAW'      
output_folder = 'SEF_output'  

os.makedirs(output_folder, exist_ok=True)

# --- Process all images in folder ---
valid_ext = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")

for filename in os.listdir(input_folder):
    if filename.lower().endswith(valid_ext):
        in_path = os.path.join(input_folder, filename)
        out_path = os.path.join(output_folder, filename)
        
        img = cv.imread(in_path)
        if img is None:
            print(f" Skipped (can't read): {filename}")
            continue
        
        neutralized = spectral_equalization_filter(img)
        cv.imwrite(out_path, neutralized)
        print(f"Saved: {out_path}")


Saved: SEF_output/RGT_0303.png
Saved: SEF_output/LFT_3414.png
Saved: SEF_output/T_S03127.png
Saved: SEF_output/T_S04915.png
Saved: SEF_output/T_S03879.png
Saved: SEF_output/T_S03037.png
Saved: SEF_output/RGT_0302.png


MODULE-2


In [8]:
from aocm_algorithm import AOCM
import cv2
import os

# input_folder = 'IlluminateNet_output'   # for low-lit
input_folder = 'SEF_output'     
output_folder = 'AOCM_output'  
os.makedirs(output_folder, exist_ok=True)

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Skipping {filename}: cannot read")
            continue
        
        stem = os.path.splitext(filename)[0]
        
        # Process and save only the final AOCM output
        AOCM(img, return_all=False, save_path=output_folder, stem=stem)
        print(f"Processed {filename}")

Processed RGT_0303.png
Processed LFT_3414.png
Processed T_S03127.png
Processed T_S04915.png
Processed T_S03879.png
Processed T_S03037.png
Processed RGT_0302.png


DEPTH ESTIMATION - Depth anything

In [9]:
from depth_generation import depth_gen

depth_gen(input_folder = "AOCM_output/",output_folder = "depths")


/home/dhatri/DIVER_git/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0
Generating PLASMA depth maps:  14%|█▍        | 1/7 [00:00<00:03,  1.66it/s]

Saved PLASMA depth for T_S03127_AOCM.png


Generating PLASMA depth maps:  29%|██▊       | 2/7 [00:01<00:02,  1.99it/s]

Saved PLASMA depth for T_S03879_AOCM.png


Generating PLASMA depth maps:  43%|████▎     | 3/7 [00:01<00:01,  2.32it/s]

...

Saved PLASMA depth for RGT_0303_AOCM.png


Generating PLASMA depth maps:  86%|████████▌ | 6/7 [00:02<00:00,  2.35it/s]

Saved PLASMA depth for T_S03037_AOCM.png


Generating PLASMA depth maps: 100%|██████████| 7/7 [00:03<00:00,  2.28it/s]

Saved PLASMA depth for T_S04915_AOCM.png


MODULE-3

In [4]:
# TRAINING
!python3 hydroopticnet_main.py --images 'AOCM_output' \
                        --depth 'depths' \
                        --output "Hydroopticnet_output" \
                        --checkpoints 'Hydroopticnet_output/check'



Seed: 1439310068774428766
training
  0%|                                                   | 0/500 [00:00<?, ?it/s]
[Veil Loss]
L1 loss (positive values): 0.163008
SmoothL1 loss (negative values): 0.012555
Total veilnet Loss: 12.717660
  0%|                                           | 1/500 [00:00<01:54,  4.35it/s]
[Veil Loss]
L1 loss (positive values): 0.162242
SmoothL1 loss (negative values): 0.012072
Total veilnet Loss: 12.234244

[Veil Loss]
L1 loss (positive values): 0.162158
SmoothL1 loss (negative values): 0.011599
Total veilnet Loss: 11.760994

[Veil Loss]
L1 loss (positive values): 0.163808
SmoothL1 loss (negative values): 0.011134
Total veilnet Loss: 11.297606
  1%|▎                                          | 4/500 [00:00<00:40, 12.37it/s]
[Veil Loss]
L1 loss (positive values): 0.166805
SmoothL1 loss (negative values): 0.010677
Total veilnet Loss: 10.844106

[Veil Loss]
L1 loss (positive values): 0.170460
SmoothL1 loss (negative values): 0.010231
Total veilnet Loss: 10.401024

In [1]:

# ## INFERENCE
!python3 hydroopticnet_inf.py --images 'AOCM_output' \
                        --depth 'depths' \
                        --output "Hydroopticnet_inf" \
                        --checkpoints 'Hydroopticnet_output/check'
